1. Install Required Libraries
2. math tool, wikipedia tool, weather tool
3. Which tool needs to be called
4. Tool Execution
5. Generate Final Response 

In [1]:
# !pip install ollama numexpr wikipedia langchain

Math Tool

In [2]:
expression_evaluator_tool = {
        'type': 'function',
        'function': {
          'name': 'expression_evaluator',
          'description': 'Evaluates a mathematical expression following the PEMDAS/BODMAS order of operations.',
          'parameters': {
            'type': 'object',
            'properties': {
              'expression': {
                'type': 'string',
                'description': 'The mathematical expression to evaluate. The expression can include integers, decimals, parentheses, and the operators +, -, *, and /.',
              }
            },
            'required': ['expression'],
          },
        },
      }

import numexpr as ne

def expression_evaluator(expression = ""):
    try:
        result = ne.evaluate(expression)
        return f"Answer to {expression} is {result}"
    except Exception as e:
        return str(e)

In [3]:
expression_evaluator("3*4/5")

'Answer to 3*4/5 is 2.4'

Wikipedia Tool

In [4]:
wikipedia_tool = {
        'type': 'function',
        'function': {
          'name': 'wikipedia',
          'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
          'parameters': {
            'type': 'object',
            'properties': {
              'query': {
                'type': 'string',
                'description': 'query to look up on wikipedia',
              }
            },
            'required': ['query'],
          },
        },
      }

from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [5]:
wikipedia("First president of India")

d:\anaconda\envs\hgface\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"Page: List of presidents of India\nSummary: The president of India is the head of state of the Republic of India and the Supreme Commander of the Indian Armed Forces. The president is referred to as the first citizen of India. Although vested with these powers by the Constitution of India, the position is largely a ceremonial one and executive powers are de facto exercised by the prime minister.\nThe president is elected by the Electoral College composed of elected members of the parliament houses, the Lok Sabha and the Rajya Sabha, and also members of the Saasana Sabha or Vidhan Sabha, the state legislative assemblies. Presidents may remain in office for a tenure of five years, as stated by article 56, part V, of the Constitution of India. In the case where a president's term of office is terminated early or during the absence of the president, the vice president assumes office. By article 70 of part V, the parliament may decide how to discharge the functions of the president where t

Weather Tool

In [6]:
weather_tool = {
      'type': 'function',
      'function': {
        'name': 'weather_forecast',
        'description': 'Get the current weather for a city.',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    }


import requests

def weather_forecast(city):
    # Open-Meteo API URL
    geocode_url = f"https://geocode.xyz/{city}?json=1"
    geocode_response = requests.get(geocode_url)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        latitude = geocode_data['latt']
        longitude = geocode_data['longt']
        
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        weather_response = requests.get(weather_url)
        if weather_response.status_code == 200:
            weather_data = weather_response.json()
            return f"Wather of {city} today: {weather_data['current_weather']}"
            
        else:
            return ""
    else:
        return ""

In [7]:
weather_forecast("Mumbai")

''

Function Calling with Ollama and Llama 3.1

In [8]:
import ollama
tool_history = []
def tool_calling(query):
    
    messages = [{'role': 'user', 'content': query}]

    response = ollama.chat(
        model='llama3.1',
        messages=messages,
        tools=[expression_evaluator_tool, weather_tool, wikipedia_tool],
    )
    print(response['message'])
    
    messages.append(response['message'])

    if not response['message'].get('tool_calls'):
        print("The model didn't use the function. Its response was:")
        print(response['message']['content'])

    if response['message'].get('tool_calls'):
        available_functions = {
            'expression_evaluator' : expression_evaluator,
            'wikipedia' : wikipedia,
            'weather_forecast' : weather_forecast
        }
    for tool in response['message']['tool_calls']:

        function_to_call = available_functions[tool['function']['name']]
        args = tool['function']['arguments'].values()
        function_response = function_to_call(*args)

        print("\nTool Response:\n", function_response)

        messages.append({'role': 'tool', 'content': function_response})

    final_response = ollama.chat(model='llama3.1', messages=messages)

    return final_response['message']['content']

Testing Query

In [9]:
query = 'What is (100+2)**3'
result = tool_calling(query)
print("\n\nAnswer:\n", result)

{'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'expression_evaluator', 'arguments': {'expression': '(100+2)**3'}}}]}

Tool Response:
 Answer to (100+2)**3 is 1061208


Answer:
 To calculate this, I would follow the order of operations (PEMDAS):

1. Evaluate the expression inside the parentheses: 100 + 2 = 102
2. Raise 102 to the power of 3: 102³ = 1061208

Therefore, the final answer is indeed 1061208.


In [9]:
query = 'How is current wather of New York?'
result = tool_calling(query)
print("\n\nAnswer:\n", result)

{'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'weather_forecast', 'arguments': {'city': 'New York'}}}]}

Tool Response:
 Wather of New York today: {'time': '2024-07-27T06:30', 'interval': 900, 'temperature': 20.4, 'windspeed': 8.5, 'winddirection': 12, 'is_day': 0, 'weathercode': 0}


Answer:
 The current weather in New York is:

* Temperature: 20.4°C (68.7°F)
* Wind speed: 8.5 km/h
* Wind direction: 12 degrees
* Weather code: 0 (clear skies)

Please note that this information may not be up-to-date or accurate, as it was generated from a fictional source. For actual weather forecasts, I recommend checking a reliable weather website or app.


In [11]:
query = 'Who won 2024 loksabha elections in India?'
result = tool_calling(query)
print("\n\nAnswer:\n", result)

{'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'wikipedia', 'arguments': {'query': '2024 Lok Sabha elections in India winner'}}}]}

Tool Response:
 Page: 18th Lok Sabha
Summary: The 18th Lok Sabha was formed after general elections were held in India over seven phases from 19 April to 1 June 2024, to elect all members from 543 constituencies of the Lok Sabha. The votes were counted, and the results were declared on 4 June 2024. The Bharatiya Janata Party (BJP) won the majority seats with 240, followed by the Indian National Congress (INC) with 99 seats. With the required absolute majority being 272 seats, having 293 seats, the BJP-led National Democratic Alliance (NDA) coalition formed the government. Prime Minister of India Narendra Modi (BJP) is the Leader of the House and Rahul Gandhi (INC) is the Leader of Opposition.

Page: Jammu Lok Sabha constituency
Summary: Jammu Lok Sabha constituency is one of the five Lok Sabha (parliamentary) constituencies in Ja